In [32]:
pserver.stop()

Closed command server
Closed parameter server


In [1]:
from parameter_server import *
pserver = ParameterServer()

pserver.add_listener("127.0.0.1", 9010)

Parameter server started on ('127.0.0.1', 9011)
Starting on port 14000
Serving on ('127.0.0.1', 14000)
Registering listener
Unhandled command [/requestListenerInfo] ~ 16987


# Usage of DiskBufferWriter

In [2]:
from disk_buffer_writer import *

In [3]:
background = DiskBufferWriter('background','background.json', 'bin/')
background.expose_to_network(pserver)

In [4]:
import random, math

background.data = [random.random() for i in range(3)]

# Cache to compensate for possible time writing overlaps

In [5]:
dataWriter = DiskBufferWriter('sine_data','output.json', 'bin/')
dataWriter.expose_to_network(pserver)
dataWriter.allow_cache(10)

In [6]:
import random, math

numpoints = random.randint(4,20)

rnums = [random.random() for i in range(15)]
rnums.sort()
dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                   'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                  }

In [7]:
dataWriter.data

{'random': [-0.9744274867551952,
  -0.6993523971753532,
  -0.6337948721611844,
  -0.4807239480996901,
  -0.41932222650401885,
  -0.3723499353161026,
  -0.04600466791337965,
  0.07039450699624528,
  0.3749721198477327,
  0.38564776778956,
  0.43049455315449836,
  0.5208577996038932,
  0.6268023426568337,
  0.7092720848724849,
  0.9051679100289123],
 'sines': [0.08025202676447478,
  0.8102111708152462,
  0.912954742822023,
  0.9981669546958482,
  0.9680514333642142,
  0.9206615494840646,
  0.14402529435623812,
  -0.21935260553681082,
  -0.9238460104597997,
  -0.9361614226195455,
  -0.9762545096878373,
  -0.9978538932065936,
  -0.921697893714255,
  -0.7915545487980213,
  -0.29353609883424725]}

In [8]:
import time
for i in range(20):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.1)

Now let's send changes really fast to test disk caching. (It may fail on your system if your drive access is not fast enough, causing the app to crash)

In [9]:
import time
dataWriter.allow_cache(100)

for i in range(200):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.001)

This might leave a large cache, so cleanup:

In [10]:
dataWriter.cleanup()

If you need guaranteed safety, use file locking:

In [11]:

# No cache but file locking
dataWriter.allow_cache(-1)
dataWriter.use_file_lock()

import time
for i in range(10000):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.00001) # Very fast updates

If this is still not enough, you can have the data writer wait for read acknowledge from app.

Currently not implemented.

In [ ]:
pserver.stop()